In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib notebook

from __code.white_beam import WhiteBeam

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Select top sample folder (containing all the runs)

In [ ]:
o_white_beam = WhiteBeam(top_sample_dir=system.System.get_working_dir())
o_white_beam.select_top_sample_folder()

In [ ]:
# Select top folder of data to reconstruct
## program will load and automatically make a list of good runs and bad runs  

# Select top OB folder (containing all the OB runs)

In [ ]:
o_white_beam.select_top_ob_folder()

# Pre-processing the data

 - looking at proton charge requested
 - looking at empty runs
 - looking at uncompatible runs (wrong proton charge, missing bins)

In [ ]:
# combine all the images within each run

In [ ]:
# pre-processing
## remove "dead" pixels
## remove abnormal high counts

In [ ]:
# normalization

In [ ]:
# chips correction

In [ ]:
# calculate center of rotation

In [ ]:
# calculate tilt correction and apply it

In [ ]:
# launch svmbir